In [49]:
import hvplot.pandas
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import requests
from config import geoapify_api_key

In [38]:
city_data_df = pd.read_csv("./weather_data/cities.csv")

city_data_df['Max Temp'] = round(((city_data_df['Max Temp'] * (9/5)) - 459.67), 2)
city_data_df.head()


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mahebourg,MU,2022-12-07 23:25:34,-20.4081,57.7000,77.29,83,75,5.66
1,1,Rikitea,PF,2022-12-07 23:25:34,-23.1203,-134.9692,75.61,78,16,8.84
2,2,Valvedditturai,LK,2022-12-07 23:25:35,9.8167,80.1667,78.37,65,100,9.30
3,3,Saint-Philippe,RE,2022-12-07 23:25:35,-21.3585,55.7679,74.79,87,38,6.98
4,4,Santa Cruz Cabralia,BR,2022-12-07 23:25:36,-16.2781,-39.0247,77.23,88,40,3.60


In [39]:
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat", 
    geo=True, 
    tiles = "OSM", 
    frame_width = 900, 
    frame_height = 500,
    size = "Max Temp",
    scale = 0.5,
    color = "City"
    )
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

In [40]:
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat", 
    geo=True, 
    tiles = "EsriNatGeo", 
    frame_width = 900, 
    frame_height = 500,
    size = "Max Temp",
    scale = 0.5,
    color = "City"
    )
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

In [41]:
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat", 
    geo=True, 
    tiles = "EsriNatGeo", 
    frame_width = 900, 
    frame_height = 500,
    size = "Wind Speed",
    scale = 3,
    color = "City"
    )
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Wind Speed)

In [46]:
min_temp = float(input("What is the minimum temperature you like for your trip?  "))
max_temp = float(input("What is the maximum temperature you like for your trip?  "))

preferred_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)
    ]
preferred_cities_df.head(10)  

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mahebourg,MU,2022-12-07 23:25:34,-20.4081,57.7000,77.29,83,75,5.66
1,1,Rikitea,PF,2022-12-07 23:25:34,-23.1203,-134.9692,75.61,78,16,8.84
2,2,Valvedditturai,LK,2022-12-07 23:25:35,9.8167,80.1667,78.37,65,100,9.30
4,4,Santa Cruz Cabralia,BR,2022-12-07 23:25:36,-16.2781,-39.0247,77.23,88,40,3.60
6,6,Atuona,PF,2022-12-07 23:21:42,-9.8000,-139.0333,77.70,77,11,8.17
10,10,Kapaa,US,2022-12-07 23:25:39,22.0752,-159.3190,84.18,79,75,8.23
19,19,Sao Filipe,CV,2022-12-07 23:25:44,14.8961,-24.4956,75.27,78,14,2.95
35,35,Bacolod,PH,2022-12-07 23:25:53,10.6667,122.9500,77.97,83,96,4.03
50,50,Kualakapuas,ID,2022-12-07 23:27:02,-3.0091,114.3876,75.43,92,96,0.91
51,51,Butaritari,KI,2022-12-07 23:27:02,3.0707,172.7902,80.11,81,74,5.60


In [48]:
hotel_df = preferred_cities_df[["City","Country","Max Temp","Lat","Lng"]].copy()
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Mahebourg,MU,77.29,-20.4081,57.7000,
1,Rikitea,PF,75.61,-23.1203,-134.9692,
2,Valvedditturai,LK,78.37,9.8167,80.1667,
4,Santa Cruz Cabralia,BR,77.23,-16.2781,-39.0247,
6,Atuona,PF,77.70,-9.8000,-139.0333,


In [56]:
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_api_key,
}

for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    
    base_url = "https://api.geoapify.com/v2/places"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"


In [61]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Mahebourg,MU,77.29,-20.4081,57.7000,Preskil Island Resort
1,Rikitea,PF,75.61,-23.1203,-134.9692,Chez Bianca & Benoit
2,Valvedditturai,LK,78.37,9.8167,80.1667,No hotel found
4,Santa Cruz Cabralia,BR,77.23,-16.2781,-39.0247,Campo Bahia
6,Atuona,PF,77.70,-9.8000,-139.0333,Pearl Resort
10,Kapaa,US,84.18,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
19,Sao Filipe,CV,75.27,14.8961,-24.4956,Bela Vista
35,Bacolod,PH,77.97,10.6667,122.9500,Seda Capitol Central
50,Kualakapuas,ID,75.43,-3.0091,114.3876,Hotel Bayu
51,Butaritari,KI,80.11,3.0707,172.7902,No hotel found


In [65]:
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Max Temp",
    scale = 1,
    color="City",
    hover_cols = ["Hotel Name", "Country"]
)
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)